# Training TF net

In [1]:
import sys
sys.path.insert(0, '/braintree/home/bashivan/dropbox/Codes/3D_fMRI_CNN/tf_pipeline/')
import os
from tf_trainer import Trainer
import logging 
from tf_model import TFModel
from tf_dataset import TFDataset
import numpy as np
import tensorflow as tf
import numpy as np
FLAGS = tf.app.flags.FLAGS
os.environ['CUDA_VISIBLE_DEVICES'] = '0,2'

fold_to_run = 0

# Load the dataset
fold_pairs = []

model = TFModel()
dataset = TFDataset(data_dir='/braintree/data2/active/users/bashivan/Data/fmri_conv')
tr = Trainer(model=model, dataset=dataset)
print("Loading data...")
tr.load_data(random=False)

sub_nums = tr.subjects
subs_in_fold = np.ceil(np.max(sub_nums) / float(10))

subs_in_fold = np.ceil(np.max(sub_nums) / float(10))
# n-fold cross validation
for i in range(FLAGS.num_folds):
    '''
    for each kfold selects fold window to collect indices for test dataset and the rest becomes train
    '''
    test_ids = np.bitwise_and(sub_nums >= subs_in_fold * (i), sub_nums < subs_in_fold * (i + 1))
    train_ids = ~ test_ids
    fold_pairs.append((np.nonzero(train_ids)[0], np.nonzero(test_ids)[0]))

    

Loading data...


In [ ]:
FLAGS = tf.app.flags.FLAGS

fold_num = 0
fold = fold_pairs[0]

print('Splitting the data...')
tr.split_data(fold)
print('Preprocessing data...')
# tr.preprocess_data()


Splitting the data...
Preprocessing data...


In [ ]:
import os
FLAGS.initial_learning_rate = 0.0001
FLAGS.batch_size = 30
FLAGS.num_gpus = 2
FLAGS.num_epochs_per_decay = 5
FLAGS.train_dir = os.path.join(FLAGS.train_dir, str(fold_num))
tr.train(fold_num=fold_num)


****************************************
subset: train
data_dir: /braintree/data2/active/users/bashivan/Data/fmri_conv_orig
num_epochs_per_decay: 5
num_gpus: 2
train_dir: /braintree/data2/active/users/bashivan/results/temp/0
batch_size: 30
model_type: lstm
num_checkpoints_tosave: 5
initial_learning_rate: 0.0001
num_examples: None
num_epochs: 10
checkpoint_dir: .
learning_rate_decay_factor: 0.1
fold_to_run: -1
num_folds: 10
seed: 0
eval_dir: /om/user/bashivan/temp
num_time_steps: 16
log_device_placement: False
****************************************
Using ADAM optimizer...
Using default loss (softmax-Xentropy)...
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
INFO:tensorflow:Summary name softmax_cross_entropy_loss/value (raw) is illegal; using softmax_cross_ent

In [9]:
model.endpoints['lstm_out'].get_shape()[0].value